## 滴滴出行A/B test（T检验）

与一般的万行A/B test实验数据不同，本实验A/B组数据各自只有29行，只能使用T检验

## 一、查看数据

数据样本由58行，8列数据组成，数据较少，各字段名称及说明如下：
- date：日期
- group：组别
- requests：订单请求数
- gmv：成交总额
- coupon per trip：每单优惠券金额
- trips：订单数
- canceled trips：取消请求数

## 二、导入库

In [4]:
import numpy as np
import matplotlib as plt
import pandas as pd

## 三、导入数据

In [5]:
df=pd.read_excel(r'F:\Data\didi4010\test.xlsx')

## 四、查看数据

In [6]:
df.head()

,date,group,requests,gmv,coupon per trip,trips,canceled requests
0,2019-01-01,control,30,7486.620,1.069746,24,1
1,2019-01-02,control,152,38301.575,1.044199,121,8
2,2019-01-03,control,267,67768.425,1.032276,215,14
3,2019-01-04,control,369,94992.900,1.028196,298,19
4,2019-01-05,control,478,123236.875,1.011807,390,24


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               58 non-null     datetime64[ns]
 1   group              58 non-null     object        
 2   requests           58 non-null     int64         
 3   gmv                58 non-null     float64       
 4   coupon per trip    58 non-null     float64       
 5   trips              58 non-null     int64         
 6   canceled requests  58 non-null     int64         
dtypes: datetime64[ns](1), float64(2), int64(3), object(1)
memory usage: 3.3+ KB


In [8]:
df.isnull().sum()

date                 0
group                0
requests             0
gmv                  0
coupon per trip      0
trips                0
canceled requests    0
dtype: int64

In [9]:
df.duplicated().sum()

0

## 五、数据清洗

数据比较理想，不用预处理，现在对后几列数据查看

In [10]:
df[df.columns[2:]]

,requests,gmv,coupon per trip,trips,canceled requests
0,30,7486.620,1.069746,24,1
1,152,38301.575,1.044199,121,8
2,267,67768.425,1.032276,215,14
3,369,94992.900,1.028196,298,19
4,478,123236.875,1.011807,390,24
5,579,148102.625,0.991845,476,28
6,867,208075.975,0.938023,714,42
7,963,233040.575,0.923250,795,47
8,1064,258725.250,0.899881,873,54
9,1183,287949.250,0.879014,965,63


In [11]:
df[df.columns[2:]].describe()

,requests,gmv,coupon per trip,trips,canceled requests
count,58.000000,5.800000e+01,58.000000,58.000000,58.000000
mean,2046.224138,4.820680e+05,0.848199,1633.551724,146.431034
std,1287.778191,2.984599e+05,0.111335,1019.612227,104.260321
min,30.000000,7.382210e+03,0.723737,24.000000,1.000000
25%,968.750000,2.336387e+05,0.756524,799.250000,45.500000
50%,2203.500000,5.092359e+05,0.790527,1745.000000,168.000000
75%,3210.750000,7.495051e+05,0.934330,2553.500000,244.000000
max,4303.000000,1.005137e+06,1.069746,3426.000000,330.000000


数据没有问题

## 六、查看group列

In [12]:
df.group.value_counts()

experiment    29
control       29
Name: group, dtype: int64

control表示的是对照组，experiment表示的是实验组，两个组都只有29行数据

## 七、添加指标

多加一列ROI(投资回报率)用于辅助我们进行判断  
因为对于电商来说，增加优惠券或者其他操作，成本近似为0，所以认为利润=收入=gmv

ROI=利润/投资=利润/(原收入-现收入)=收入/(订单数* 每单优惠数)

In [13]:
df['ROI']=df.gmv/(df['coupon per trip']*df.trips)

In [14]:
# 查看数据
df.head()

,date,group,requests,gmv,coupon per trip,trips,canceled requests,ROI
0,2019-01-01,control,30,7486.620,1.069746,24,1,291.604269
1,2019-01-02,control,152,38301.575,1.044199,121,8,303.143166
2,2019-01-03,control,267,67768.425,1.032276,215,14,305.346684
3,2019-01-04,control,369,94992.900,1.028196,298,19,310.026761
4,2019-01-05,control,478,123236.875,1.011807,390,24,312.304607


## 八、requests 假设检验

由于样本数<30，选用t检验对requests进行检验

由于设置了对照组和实验组，我认为两组对象应该是独立对象，对独立对象进行t检验需要先检验其是否齐方差

### 齐方差检验

#### 零假设

令对照组方差为c1，实验组方差为c2  
零假设H<sub>0</sub>：c1=c2  
备择假设H<sub>1</sub>：c1<>c2

#### 判断方差齐性

In [15]:
requests_A=df[df.group=='control'].requests
requests_A

0       30
1      152
2      267
3      369
4      478
5      579
6      867
7      963
8     1064
9     1183
10    1289
11    1404
12    1540
13    2098
14    2207
15    2316
16    2420
17    2505
18    2600
19    2718
20    3116
21    3220
22    3321
23    3419
24    3503
25    3608
26    3708
27    4201
28    4303
Name: requests, dtype: int64

In [16]:
requests_B=df[df.group=='experiment'].requests
requests_B

29      30
30     152
31     277
32     383
33     497
34     602
35     886
36     986
37    1083
38    1201
39    1312
40    1421
41    1555
42    2094
43    2200
44    2307
45    2410
46    2493
47    2588
48    2702
49    3084
50    3183
51    3288
52    3386
53    3470
54    3568
55    3662
56    4157
57    4256
Name: requests, dtype: int64

In [17]:
import scipy.stats as st
st.levene(requests_A,requests_B)

LeveneResult(statistic=0.014685075667736849, pvalue=0.903980667108546)

因为pvalue=0.904>α=0.05，我们接受H<sub>0</sub>，拒绝H<sub>1</sub>  
所以我们认为c1=c2，也就是两者是齐方差

### requests检验

查看两者均值

In [18]:
requests_A.mean()

2049.9310344827586

In [19]:
requests_B.mean()

2042.5172413793102

#### 零假设

令A组均值为u1，B组均值为u2

H<sub>0</sub>：u1=u2  
H<sub>1</sub>：u1<>u2

#### 确定假设检验方向

使用双尾

#### 确定检验类型和统计量

由之前的过程，我们使用t检验，并且t检验为齐方差检验

#### 确定显著性水平α

令α=0.05

#### 假设检验过程

In [20]:
# 导入库
from scipy.stats import ttest_ind

In [21]:
ttest_ind(requests_A,requests_B)

Ttest_indResult(statistic=0.021729103766715982, pvalue=0.9827412881046896)

得到p=0.983>α，接受H<sub>0</sub>，拒绝H<sub>1</sub>

我们得到结论，实验条件对requests影响不大

## 九、gmv检验

由于样本数<30，选用t检验对gmv进行检验

由于设置了对照组和实验组，我认为两组对象应该是独立对象，对独立对象进行t检验需要先检验其是否齐方差

### 齐方差检验

#### 零假设

令对照组方差为c1，实验组方差为c2  
零假设H0：c1=c2  
备择假设H1：c1<>c2

#### 判断方差齐性

In [22]:
requests_A=df[df.group=='control'].gmv
requests_A

0        7486.620
1       38301.575
2       67768.425
3       94992.900
4      123236.875
5      148102.625
6      208075.975
7      233040.575
8      258725.250
9      287949.250
10     314034.500
11     343593.750
12     375234.000
13     482983.250
14     511808.000
15     540765.250
16     568005.750
17     591686.750
18     616598.750
19     646505.250
20     726422.250
21     753117.250
22     780021.500
23     804938.250
24     828042.750
25     854086.250
26     877773.250
27     980150.500
28    1005137.000
Name: gmv, dtype: float64

In [23]:
requests_B=df[df.group=='experiment'].gmv
requests_B

29      7382.210
30     36580.200
31     67626.375
32     94601.700
33    123388.850
34    149430.600
35    209840.625
36    235433.200
37    260928.500
38    290042.250
39    318077.000
40    345412.750
41    375812.250
42    479094.250
43    506663.750
44    535324.000
45    562010.500
46    584459.250
47    609191.000
48    636740.500
49    713343.750
50    738668.500
51    764081.750
52    789127.250
53    812143.250
54    837921.750
55    861041.750
56    960759.000
57    986233.750
Name: gmv, dtype: float64

In [24]:
import scipy.stats as st
st.levene(requests_A,requests_B)

LeveneResult(statistic=0.02865341299111212, pvalue=0.8661917430097603)

因为pvalue=0.866>α=0.05，我们接受H<sub>0</sub>，拒绝H<sub>1</sub>  
所以我们认为c1=c2，也就是两者是齐方差

###  gmv检验

查看两者均值

In [25]:
requests_A.mean()

485123.5972413793

In [26]:
requests_B.mean()

479012.4313793103

#### 零假设

令A组均值为u1，B组均值为u2

H<sub>0</sub>：u1=u2

H<sub>1</sub>：u1<>u2

#### 确定假设检验方向

使用双尾

#### 确定检验类型和统计量

由之前的过程，我们使用t检验，并且t检验为齐方差检验

#### 确定显著性水平α

令α=0.05

#### 假设检验过程

In [29]:
# 导入库
from scipy.stats import ttest_ind

In [30]:
ttest_ind(requests_A,requests_B)

Ttest_indResult(statistic=0.07728616692818668, pvalue=0.9386713733961323)

得到p=0.938>α，接受H<sub>0</sub>，拒绝H<sub>1</sub>

我们得到结论，实验条件对gmv影响不大

#### 误差分析

注：如果把实验对象当作相关性对象检测，结果为：

In [33]:
st.ttest_rel(requests_A,requests_B)

Ttest_relResult(statistic=4.247583846321442, pvalue=0.00021564303983362577)

这里的0.0002更加合理，但是这里我将两个数据当作独立数据考虑，进行t检验

## 十、ROI检验

由于样本数<30，选用t检验对gmv进行检验

由于设置了对照组和实验组，我认为两组对象应该是独立对象，对独立对象进行t检验需要先检验其是否齐方差

### 齐方差检验

#### 零假设

令对照组方差为c1，实验组方差为c2  
零假设H<sub>0</sub>：c1=c2  
备择假设H<sub>1</sub>：c1<>c2  

#### 判断方差齐性

In [37]:
requests_A=df[df.group=='control'].ROI
requests_A

0     291.604269
1     303.143166
2     305.346684
3     310.026761
4     312.304607
5     313.698098
6     310.677730
7     317.500829
8     329.336306
9     339.463217
10    352.132476
11    357.554820
12    363.775826
13    385.067775
14    391.205131
15    398.281549
16    404.525992
17    410.141385
18    413.174493
19    407.307217
20    391.758812
21    389.380259
22    386.483110
23    385.207786
24    383.900057
25    381.202612
26    378.373959
27    370.713305
28    367.364146
Name: ROI, dtype: float64

In [39]:
requests_B=df[df.group=='experiment'].ROI
requests_B

29    295.960040
30    285.160963
31    290.451626
32    295.171509
33    296.395066
34    301.253014
35    301.318100
36    307.780638
37    317.806505
38    327.485256
39    338.666227
40    346.653174
41    352.018340
42    377.564021
43    384.205494
44    391.816340
45    396.859893
46    401.959183
47    405.536343
48    400.086547
49    384.622243
50    382.648953
51    380.367029
52    378.386379
53    377.873886
54    375.623503
55    373.809886
56    366.901602
57    364.364529
Name: ROI, dtype: float64

In [40]:
import scipy.stats as st
st.levene(requests_A,requests_B)

LeveneResult(statistic=0.10363884745479335, pvalue=0.7487044961896305)

因为pvalue=0.748>α=0.05，我们接受H<sub>0</sub>，拒绝H<sub>1</sub>  
所以我们认为c1=c2，也就是两者是齐方差

#### ROI检验

查看两者均值

In [41]:
requests_A.mean()

360.3673233425277

In [42]:
requests_B.mean()

351.68090647475714

#### 零假设

令A组均值为u1，B组均值为u2

H<sub>0</sub>：u1=u2

H<sub>1</sub>：u1<>u2

#### 确定假设检验方向

使用双尾

#### 确定检验类型和统计量

由之前的过程，我们使用t检验，并且t检验为齐方差检验

#### 确定显著性水平α

令α=0.05

#### 假设检验过程

In [46]:
# 导入库
from scipy.stats import ttest_ind

In [47]:
ttest_ind(requests_A,requests_B)

Ttest_indResult(statistic=0.8430996450057455, pvalue=0.40275951535453325)

得到p=0.402>α=0.05，接受H<sub>0</sub>，拒绝H<sub>1</sub>  
我们得到结论，实验条件对ROI影响不大

## 十一、结论

经过我查找资料之后，结论如下：  
配对检验是基于对同一样本中相同个体的多次测量数据的检验  
独立检验是对于不同样本的个体的测量数据

本实验的结果比较不合理，原因在于数据集中并没有注明两组实验是配对样本还是独立样本  
我理解的样本是A/B组独立进行实验，所以结论都建立在两者为独立样本的基础上进行的

所以实验结果显示，行为措施对ROI影响不大